In [1]:
from dotenv import load_dotenv
import os
import csv
import requests
from sec_api import QueryApi
from lxml import etree
from pathlib import Path
import pandas as pd
from tqdm import tqdm

load_dotenv()

True

In [ ]:
# Retrieve all NVIDIA-related insider trading (Form 4) filings
# filed between 2024-01-01 and 2025-06-30 using SEC-API.


# -------------------------------------------------------------------
# Configuration
# -------------------------------------------------------------------

API_KEY = os.environ.get("SEC_API_KEY")
if not API_KEY:
    raise RuntimeError("Set the SEC_API_KEY environment variable with your SEC-API key.")

OUT_CSV = "data/sec_filing/nvda_form4_secapi_20240101_20250630.csv"

# Lucene-like SEC-API query
# This fetches all Form 4 filings mentioning NVIDIA (company, ticker, or text)
# within the given date range.
lucene_query = (
    'formType:"4" AND filedAt:[2024-01-01 TO 2025-06-30] AND (content:"NVIDIA" OR ticker:NVDA OR companyName:"NVIDIA")'
)


payload = {
    "query": lucene_query,
    "from": "0",  # starting offset
    "size": "100",  # number of filings per page (100 is stable)
    "sort": [{"filedAt": {"order": "asc"}}],
}

# -------------------------------------------------------------------
# Fetching all filings (handles pagination)
# -------------------------------------------------------------------


def fetch_all_filings():
    query_api = QueryApi(api_key=API_KEY)
    all_filings = []
    offset = 0
    page_size = int(payload["size"])

    while True:
        payload["from"] = str(offset)
        response = query_api.get_filings(payload)

        # Defensive parsing
        filings = response.get("filings", [])
        if not filings:
            break

        all_filings.extend(filings)
        offset += len(filings)

        # Stop if last page
        if len(filings) < page_size:
            break

    return all_filings


# -------------------------------------------------------------------
# Main execution
# -------------------------------------------------------------------


def main():
    filings = fetch_all_filings()

    # Expected keys returned by SEC-API
    keys = [
        "filedAt",
        "formType",
        "companyName",
        "ticker",
        "cik",
        "accessionNo",
        "linkToFilingDetails",
    ]

    with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for fobj in filings:
            writer.writerow({k: fobj.get(k, "") for k in keys})

    print(f"✅ Wrote {len(filings)} filings to {OUT_CSV}")

In [4]:
main()

✅ Wrote 170 filings to nvda_form4_secapi_20240101_20250630.csv


In [2]:
INPUT_CSV = "data/sec_filing/nvda_form4_secapi_20240101_20250630.csv"
CACHE_DIR = Path("data/sec_filing/xml_cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)
USER_AGENT = "Your Name your_email@example.com"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
df_meta = pd.read_csv(INPUT_CSV)
urls = df_meta["linkToFilingDetails"].dropna().unique()
urls

array(['https://www.sec.gov/Archives/edgar/data/1045810/000104581024000004/xslF345X05/wk-form4_1704840790.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000006/xslF345X05/wk-form4_1704840872.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000009/xslF345X05/wk-form4_1705099343.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000011/xslF345X05/wk-form4_1705444387.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000037/xslF345X05/wk-form4_1709071330.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000038/xslF345X05/wk-form4_1709071452.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000040/xslF345X05/wk-form4_1709071584.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000041/xslF345X05/wk-form4_1709071698.xml',
       'https://www.sec.gov/Archives/edgar/data/1045810/000122520824003814/xslF345X05/doc4.xml',
       

In [4]:
def download_xml(url, folder=CACHE_DIR):
    """Download one SEC Form 4 XML to local cache."""
    fname = folder / url.split("/")[-1]
    if fname.exists():
        return  # already cached
    headers = {"User-Agent": USER_AGENT}
    try:
        r = requests.get(url, headers=headers, timeout=30)
        r.raise_for_status()
        fname.write_bytes(r.content)
    except Exception as e:
        print(f"Download failed: {url} | {e}")


# Download all files (only new ones)
for url in tqdm(urls, desc="Downloading XMLs"):
    download_xml(url)

print(f"✅ Cached {len(list(CACHE_DIR.glob('*.xml')))} XML files in {CACHE_DIR}")

✅ Cached 165 XML files in data\sec_filing\xml_cache


In [39]:
from pathlib import Path
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import re
from datetime import datetime

# ----------------------------
# Safe text + cleaning
# ----------------------------
_ws = re.compile(r"\s+")
_commas = re.compile(r",")
_money = re.compile(r"^\$?\s*")


def _safe_text(node) -> str:
    if node is None:
        return ""
    if isinstance(node, (Tag, BeautifulSoup)):
        return node.get_text(" ", strip=True)
    if isinstance(node, NavigableString):
        return str(node).strip()
    return str(node).strip()


def _clean(txt: str) -> str:
    return _ws.sub(" ", txt).strip() if txt else ""


def _num_or_none(txt: str):
    t = _clean(txt)
    if not t:
        return None
    t = _commas.sub("", t)
    t = _money.sub("", t)
    try:
        return float(t) if "." in t else int(t)
    except ValueError:
        return None


def _first_text_or_empty(el):
    return _clean(_safe_text(el))


# ----------------------------
# HTML branch helpers
# ----------------------------
def _find_after_label(soup, label_substring: str):
    lab = soup.find(
        lambda t: isinstance(t, Tag) and t.name in ("td", "span") and label_substring in _first_text_or_empty(t)
    )
    if not lab:
        return ""
    nxt = lab.find_next(lambda t: isinstance(t, Tag) and t.name == "span" and "FormData" in (t.get("class") or []))
    return _first_text_or_empty(nxt)


def _extract_footnotes_html(soup):
    footnotes = {}

    def is_expl(t):
        return (
            isinstance(t, Tag)
            and t.name in ("td", "div", "th", "span")
            and "Explanation of Responses" in _first_text_or_empty(t)
        )

    for hdr in soup.find_all(is_expl):
        container = hdr if isinstance(hdr, Tag) else hdr.parent
        if not container:
            continue
        parent = container.find_parent() or container
        for td in parent.find_all("td", class_=lambda c: c and "FootnoteData" in c.split()):
            text = _first_text_or_empty(td)
            m = re.match(r"^\s*\(?(\d+)\)?\.\s*(.*)$", text)
            if m:
                footnotes[m.group(1)] = m.group(2).strip()
    return footnotes


def _collect_cell_footnotes(cell) -> list:
    if not isinstance(cell, Tag):
        return []
    nums = []
    for sup in cell.find_all("sup"):
        s = _first_text_or_empty(sup)
        nums.extend(re.findall(r"\(?(\d+)\)?", s))
    return nums


def _extract_issuer_and_ticker_html(soup):
    issuer_cell = soup.find(
        lambda t: isinstance(t, Tag) and t.name in ("td", "span") and "Issuer Name" in _first_text_or_empty(t)
    )
    issuer = ticker = ""
    if issuer_cell:
        link = issuer_cell.find_next("a")
        issuer = _first_text_or_empty(link) or _first_text_or_empty(issuer_cell.find_next("span", class_="FormData"))
        # Try bracketed ticker [ NVDA ]
        scope = issuer_cell.parent if isinstance(issuer_cell, Tag) else soup
        txt = _first_text_or_empty(scope)
        m = re.search(r"\[\s*([A-Za-z.\-]+)\s*\]", txt)
        if m:
            ticker = m.group(1)
    return issuer, ticker


def _extract_reporting_person_html(soup):
    name_hdr = soup.find(
        lambda t: isinstance(t, Tag)
        and t.name in ("td", "span")
        and "Name and Address of Reporting Person" in _first_text_or_empty(t)
    )
    if not name_hdr:
        return ""
    link = name_hdr.find_next("a")
    return _first_text_or_empty(link)


def _extract_10b5_1_flag_html(soup):
    label = "Check this box to indicate that a transaction was made pursuant to a contract, instruction or written plan"
    lab = soup.find(lambda t: isinstance(t, Tag) and t.name in ("td", "span") and label in _first_text_or_empty(t))
    if not lab:
        return False
    box_td = lab.find_previous("td")
    return "X" in _first_text_or_empty(box_td)


def _find_table_i_html(soup):
    hdr = soup.find(
        lambda t: isinstance(t, Tag)
        and t.name in ("th", "td", "span")
        and "Table I - Non-Derivative Securities" in _first_text_or_empty(t)
    )
    if not hdr:
        return None
    thead = hdr.find_parent("thead")
    if thead:
        table = thead.find_parent("table")
        if table:
            return table.find("tbody") or table
    table = hdr.find_parent("table")
    return (table.find("tbody") or table) if table else None


# ----------------------------
# XML branch (ownershipDocument)
# ----------------------------
def _is_xml_form4(soup) -> bool:
    return bool(soup.find(lambda t: isinstance(t, Tag) and t.name.lower().endswith("ownershipdocument")))


def _tag(soup, path):
    """Shorthand to find first tag by exact name anywhere."""
    return soup.find(lambda t: isinstance(t, Tag) and t.name.lower().endswith(path.lower()))


def _xml_text(parent, path):
    if not parent:
        return ""
    el = parent.find(lambda t: isinstance(t, Tag) and t.name.lower().endswith(path.lower()))
    return _first_text_or_empty(el)


def _parse_xml_table_i(soup, source_file):
    doc = _tag(soup, "ownershipDocument")
    if not doc:
        return pd.DataFrame()
    # Metadata
    issuer = _xml_text(doc, "issuerName")
    ticker = _xml_text(doc, "issuerTradingSymbol") or _xml_text(doc, "issuerSymbol")
    rper = _xml_text(doc, "reportingOwner/reportingOwnerId/rptOwnerName")
    earliest = _xml_text(doc, "periodOfReport")
    tenb5 = False  # XML has a specific checkbox in some schemas; often captured via 'nonDerivativeTable' footnotes; default False

    rows = []
    nd_table = doc.find(lambda t: isinstance(t, Tag) and t.name.lower().endswith("nonDerivativeTable"))
    if not nd_table:
        return pd.DataFrame()
    for ntx in nd_table.find_all(lambda t: isinstance(t, Tag) and t.name.lower().endswith("nonDerivativeTransaction")):
        title = _xml_text(ntx, "securityTitle/value")
        tx_date = _xml_text(ntx, "transactionDate/value")
        code = _xml_text(ntx, "transactionCoding/transactionCode")
        code_v = _xml_text(ntx, "transactionCoding/transactionFormType")  # rarely used; keep for parity
        amount = _num_or_none(_xml_text(ntx, "transactionAmounts/transactionShares/value"))
        a_or_d = _xml_text(ntx, "transactionAmounts/transactionAcquiredDisposedCode/value")
        price = _num_or_none(_xml_text(ntx, "transactionAmounts/transactionPricePerShare/value"))
        owned_after = _num_or_none(_xml_text(ntx, "postTransactionAmounts/sharesOwnedFollowingTransaction/value"))
        ownership_form = _xml_text(ntx, "ownershipNature/directOrIndirectOwnership/value")
        indirect_nature = _xml_text(ntx, "ownershipNature/natureOfOwnership")
        rows.append(
            {
                "source_file": source_file,
                "issuer": issuer,
                "ticker": ticker,
                "reporting_person": rper,
                "date_earliest_transaction": earliest,
                "tenb5_1_checked": tenb5,
                "security_title": title,
                "transaction_date": tx_date,
                "transaction_code": code,
                "code_v_flag": code_v,
                "shares_amount": amount,
                "action_A_or_D": a_or_d,
                "price": price,
                "owned_after": owned_after,
                "ownership_form": ownership_form,
                "indirect_nature": indirect_nature,
                "fn_refs": {},
                "fn_texts": {},
            }
        )
    return pd.DataFrame(rows)


# ----------------------------
# Core parser
# ----------------------------
def parse_form4_file(path: Path) -> pd.DataFrame:
    with open(path, "rb") as f:
        data = f.read()
    soup = BeautifulSoup(data, "lxml")
    if not soup or not soup.find():  # fallback
        soup = BeautifulSoup(data, "html.parser")

    # XML branch
    if _is_xml_form4(soup):
        df = _parse_xml_table_i(soup, path.name)
        return df

    # HTML branch
    issuer, ticker = _extract_issuer_and_ticker_html(soup)
    reporting_person = _extract_reporting_person_html(soup)
    earliest_tx_date = _find_after_label(soup, "3. Date of Earliest Transaction")
    tenb5_1 = _extract_10b5_1_flag_html(soup)
    footnotes = _extract_footnotes_html(soup)

    table_body = _find_table_i_html(soup)
    rows_out = []

    if isinstance(table_body, Tag):
        for tr in table_body.find_all("tr"):
            tds = tr.find_all("td")
            if len(tds) < 5:
                continue

            def cell(i):
                return tds[i] if i < len(tds) else None

            title = _first_text_or_empty(cell(0))
            if not title:
                continue

            tx_date = _first_text_or_empty(cell(1))
            code = _first_text_or_empty(cell(3))
            code_v_flag = _first_text_or_empty(cell(4))
            amount = _num_or_none(_first_text_or_empty(cell(5)))
            a_or_d = _first_text_or_empty(cell(6))
            price = _num_or_none(_first_text_or_empty(cell(7)))
            owned_after = _num_or_none(_first_text_or_empty(cell(8)))
            ownership_form = _first_text_or_empty(cell(9))
            indirect_nature = _first_text_or_empty(cell(10))

            fn_refs = {
                "code": _collect_cell_footnotes(cell(3)),
                "price": _collect_cell_footnotes(cell(7)),
                "owned_after": _collect_cell_footnotes(cell(8)),
                "indirect": _collect_cell_footnotes(cell(10)),
            }
            fn_texts = {k: [footnotes.get(n, "") for n in nums if n in footnotes] for k, nums in fn_refs.items()}

            rows_out.append(
                {
                    "source_file": path.name,
                    "issuer": issuer,
                    "ticker": ticker,
                    "reporting_person": reporting_person,
                    "date_earliest_transaction": earliest_tx_date,
                    "tenb5_1_checked": tenb5_1,
                    "security_title": title,
                    "transaction_date": tx_date,
                    "transaction_code": code,
                    "code_v_flag": code_v_flag,
                    "shares_amount": amount,
                    "action_A_or_D": a_or_d,
                    "price": price,
                    "owned_after": owned_after,
                    "ownership_form": ownership_form,
                    "indirect_nature": indirect_nature,
                    "fn_refs": fn_refs,
                    "fn_texts": fn_texts,
                }
            )

    df = pd.DataFrame(rows_out)

    # Normalize dates to ISO when they look like dates
    def _to_iso(x):
        if not isinstance(x, str):
            return x
        s = x.strip()
        for fmt in ("%m/%d/%Y", "%m-%d-%Y", "%Y-%m-%d"):
            try:
                return datetime.strptime(s, fmt).date().isoformat()
            except Exception:
                pass
        return s

    if not df.empty:
        for col in ("transaction_date", "date_earliest_transaction"):
            if col in df.columns:
                df[col] = df[col].map(_to_iso)
    return df


def parse_all_form4(dir_path: str = r"data\sec_filing\xml_cache") -> pd.DataFrame:
    base = Path(dir_path)
    all_rows = []
    for p in sorted(base.glob("*.xml")):
        try:
            df = parse_form4_file(p)
            if df.empty:
                # emit a visible but non-crashing record
                all_rows.append(pd.DataFrame([{"source_file": p.name, "parse_error": "no Table I rows found"}]))
            else:
                all_rows.append(df)
        except Exception as e:
            all_rows.append(pd.DataFrame([{"source_file": p.name, "parse_error": str(e)}]))
    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


df = parse_all_form4(CACHE_DIR)
df = df.drop(columns=["fn_refs", "fn_texts"])
df.to_csv(OUT_CSV, index=False)
df

C:\Users\promb\AppData\Local\Temp\ipykernel_14836\2927998573.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


,source_file,issuer,ticker,reporting_person,date_earliest_transaction,tenb5_1_checked,security_title,transaction_date,transaction_code,code_v_flag,shares_amount,action_A_or_D,price,owned_after,ownership_form,indirect_nature,parse_error
0,doc4.xml,NVIDIA CORP,NVDA,COXE TENCH,2024-03-05,False,Common,2024-03-05,S,,1000.0,D,NaN,3296136.0,I,By Trust (2),NaN
1,doc4.xml,NVIDIA CORP,NVDA,COXE TENCH,2024-03-05,False,Common,2024-03-05,S,,9590.0,D,NaN,3286546.0,I,By Trust (2),NaN
2,doc4.xml,NVIDIA CORP,NVDA,COXE TENCH,2024-03-05,False,Common,2024-03-05,S,,189410.0,D,NaN,3097136.0,I,By Trust (2),NaN
3,doc4.xml,NVIDIA CORP,NVDA,COXE TENCH,2024-03-05,False,Common,,,,NaN,,NaN,5228.0,D,,NaN
4,doc4.xml,NVIDIA CORP,NVDA,COXE TENCH,2024-03-05,False,Common,,,,NaN,,NaN,685248.0,I,By Profit Sharing Plan Trust (5),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,wk-form4_1751317362.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2025-06-26,False,Common Stock,,,,NaN,,NaN,16070550.0,I,By the Envy Trust (3),NaN
1163,wk-form4_1751317475.xml,NVIDIA CORP,NVDA,Ochoa Ellen,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,4968.0,D,,NaN
1164,wk-form4_1751317588.xml,NVIDIA CORP,NVDA,HUDSON DAWN E,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,458887.0,D,,NaN
1165,wk-form4_1751317746.xml,NVIDIA CORP,NVDA,JONES HARVEY C,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,70407.0,D,,NaN


In [46]:
df_ok = df[df["parse_error"].isna()].copy()

# 2) filter to real trades by code and direction
target_codes = {"A", "D", "S", "P"}
mask = (
    df_ok["transaction_code"].isin(target_codes)
    & df_ok["action_A_or_D"].isin(["A", "D"])
    & df_ok["shares_amount"].fillna(0).gt(0)  # exclude balances and zero-qty rows
)

trades = df_ok[mask].copy()

# Optional: if you only want executions with an actual price on the ticket
trades_priced = trades[trades["price"].notna()].copy()

# Nice-to-have hygiene
trades["transaction_date"] = pd.to_datetime(trades["transaction_date"], errors="coerce")
trades = trades.sort_values(["transaction_date", "source_file"]).reset_index(drop=True)
trades

,source_file,issuer,ticker,reporting_person,date_earliest_transaction,tenb5_1_checked,security_title,transaction_date,transaction_code,code_v_flag,shares_amount,action_A_or_D,price,owned_after,ownership_form,indirect_nature,parse_error
0,wk-form4_1709071452.xml,NVIDIA CORP,NVDA,MCCAFFERY MICHAEL G,2023-12-13,False,Common Stock,2023-12-13,S,,4250.0,D,481.9877,17566.0,D,,NaN
1,wk-form4_1705099343.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2024-01-10,False,Common Stock,2024-01-10,S,,18000.0,D,NaN,1206731.0,I,By Trust (2),NaN
2,wk-form4_1705099343.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2024-01-10,False,Common Stock,2024-01-10,S,,3800.0,D,542.5000,1202931.0,I,By Trust (2),NaN
3,wk-form4_1705099343.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2024-01-10,False,Common Stock,2024-01-10,S,,5000.0,D,543.5000,1197931.0,I,By Trust (2),NaN
4,wk-form4_1705099343.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2024-01-10,False,Common Stock,2024-01-10,S,,8848.0,D,545.5000,1189083.0,I,By Trust (2),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,wk-form4_1751317235.xml,NVIDIA CORP,NVDA,Shah Aarti S.,2025-06-26,False,Common,2025-06-26,A,,1799.0,A,NaN,55007.0,D,,NaN
128,wk-form4_1751317362.xml,NVIDIA CORP,NVDA,STEVENS MARK A,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,11543401.0,D,,NaN
129,wk-form4_1751317475.xml,NVIDIA CORP,NVDA,Ochoa Ellen,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,4968.0,D,,NaN
130,wk-form4_1751317588.xml,NVIDIA CORP,NVDA,HUDSON DAWN E,2025-06-26,False,Common Stock,2025-06-26,A,,1799.0,A,NaN,458887.0,D,,NaN
